# 모태솔로를 위한 소개팅 연습 봇

### 라이브러리 임포트

In [27]:
from dotenv import load_dotenv
from openai import OpenAI
import speech_recognition as sr
from pydub import AudioSegment
from pydub.playback import play
import io, json, os, time
import tempfile, subprocess #맥 afplay 사용

In [28]:
#!pip install pydub

In [29]:
#!pip install SpeechRecognition

In [30]:
#!pip install openai

In [31]:
#!brew install portaudio

### 1. open API 로드

In [32]:
load_dotenv()
client = OpenAI()

______________

### 2. STT (음성 -> 텍스트 변환 함수)

In [ ]:
def stt(language: str = "ko-KR"):
    recognizer = sr.Recognizer()
    print(" STT 시작 (종료: '종료')")

    with sr.Microphone() as source:
        print("🎙 주변 소음 보정 중...")
        recognizer.adjust_for_ambient_noise(source, duration=0.5)
        print("준비 완료! '남자' 또는 '여자'로 시작해도 되고, '종료'라고 말하면 끝납니다.")

        while True:
            print("\n말씀하세요...")
            audio = recognizer.listen(source)  # 녹음

            # 인식 실패는 그냥 넘어가고 계속
            try:
                txt = recognizer.recognize_google(audio, language=language)
            except:
                continue

            print(f" 입력 내용 : {txt}")

            if txt == "종료":
                print("============================빠이염========================")
                break
            
            global user_gender, chatbot_voice
            if user_gender is None:
                if txt.strip() == "남자":
                    user_gender = "남자"      # 사용자 성별
                    # 사용자=남자 → 상대는 여자(옥순) → 챗봇은 '남자 목소리'
                    chatbot_voice = "alloy"  
                elif txt.strip() == "여자":
                    user_gender = "여자"
                    chatbot_voice = "nova"    

            
            messages.append({"role": "user", "content": txt})

            
            gpt_json = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                temperature=0.7,
                max_tokens=512
            ).choices[0].message.content

            # 상대방 대사만 추출 → 출력/재생
            partner_line = extract_partner_reply(gpt_json)
            print("상대방:", partner_line)
            tts_play(partner_line, voice=chatbot_voice)


            messages.append({"role": "assistant", "content": gpt_json})

### 3. TTS(텍스트 -> 음성변환 함수)

In [34]:
def play_wav_bytes(wav_bytes: bytes):
    audio = AudioSegment.from_file(io.BytesIO(wav_bytes), format="wav")
    play(audio)

In [35]:
def tts_play(text: str, voice: str = "nova"):
    with client.audio.speech.with_streaming_response.create(
        model="tts-1",
        voice=voice,
        input=text
    ) as resp:
        mp3_bytes = b"".join(resp.iter_bytes())

    f = tempfile.NamedTemporaryFile(suffix=".mp3", delete=False)
    f.write(mp3_bytes)
    f.flush(); f.close()
    subprocess.run(["afplay", f.name])   # macOS 기본 오디오 플레이어
    os.remove(f.name)

In [36]:
#tts_play("안녕하세요, 이 테스트 음성이 잘 들리면 afplay 성공!", voice="nova")

### 4. 프롬포트 

In [ ]:
def extract_partner_reply(json_text: str) -> str:
    data = json.loads(json_text)  
    lst = data["json_list"]
    last = lst[-1]

    # '상대방' -> '옥순' -> '상철' 순서로 찾기
    for key in ("상대방", "옥순", "상철"):
        if key in last:
            return str(last[key]).strip()

    # 마지막 턴에 없으면 역순으로 탐색
    for turn in reversed(lst):
        for key in ("상대방", "옥순", "상철"):
            if key in turn:
                return str(turn[key]).strip()

    # 못 찾으면 원문 반환
    return json_text.strip()

In [ ]:
system_prompt = """
너는 모태솔로의 소개팅 연습을 도와주는 가상의 소개팅 상대방이다.
이성과 대화한 경험이 적고, 이성과 대화를 수줍어하는 모태솔로를 위해 소개팅연습을 해준다.

[역할/캐릭터 규칙]
1) 대화 시작에 사용자에게 성별 선택지를 제시한다: "남자 / 여자".
2) 모든 대화는 정중한 존댓말로 한다. 
2) 사용자가 "남자"를 선택하면, 너는 여자 '옥순'으로 설정하고 애교 있는 말투로 말한다.
   예시) 아잉, 좋앙? , 했또, 띠드버거 같은 애교있는 말투 
3) 사용자가 "여자"를 선택하면, 너는 남자 '상철'로 설정하고 듬직하게 리드하는 말투로 말한다.
4) 처음에는 간단한 인사만 한다.
5) 이후, 본격적으로 질문을 시작한다. 질문 순서: (1) 나이/사는 곳 → (2) 취미 → (3) 연애 스타일 → (4) 자유 대화. 
6) 질문 순서별로 대화를 넘어갈때는 자연스럽게 넘어간다. 필요하다면 말을 한번 더 할수도 있다.
   매 턴마다: 짧은 리액션 + 다음 질문 1개만.
7) 중간에, 사용자가 넌센스 개그또는 아재개그를 하면 부정적인 반응(실망스러운) 리액션을 한다. 
8) 총 5~6턴 후 마무리하고, 마지막에 피드백을 제공한다:
   - 장점, 개선점, 자연스러움 점수(10점 만점)
   - 추천 에프터 멘트 1개

[출력 형식]
- 반드시 순수 JSON만 반환한다(설명/코드블록 금지). 최상위에 "json_list" 키만 존재해야 한다.
- "json_list" 하위에는 대화 턴별 객체들이 순서대로 포함된다.
- 각 객체는 "User"와 "상대방" 두 개의 키를 가지며,
  "상대방" 키 값은 실제 캐릭터 이름(옥순 또는 상철)으로 바꿔 사용해도 된다.
  (예: "옥순": "..." 또는 "상철": "...")
- 대화 5번이 끝나면, 피드백을 알려준다. 
"""

messages = [{"role": "system", "content": system_prompt}]

user_gender = None           # 사용자가 말한 성별: "남자" 또는 "여자"
chatbot_voice = "alloy"      # 기본값(남자톤)

### 5. 실행

마이크 입력 (stt) → GPT(JSON) → extract_partner_reply → TTS 변환/재생